## Callin Switzer
April 2019

Multiprocessing with multiple arguments

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import seaborn as sns
from scipy.integrate import odeint
import random
import time
from datetime import datetime
import sys
from multiprocessing import Pool, cpu_count
import simUtils # note that this is a custom-written file 
import simTest

print(sys.version)

In [ ]:
now = datetime.now()
print("last run on " + str(now))


In [ ]:
import importlib
import functools


In [ ]:
np.random.seed(123)

In [ ]:
globalDict = {"bhead": 0.507,
            "ahead": 0.908,
            "bbutt": 0.1295,
            "abutt": 1.7475, 
            "rho": 1, 
            "rhoA": 0.00118, 
            "muA": 0.000186, 
            "L1": 0.908, 
            "L2": 1.7475,  
            "L3": 0.75,
            "K": 29.3,
            "c":  14075.8,
            "g": 980.0,
            "betaR":  0.0,
            "nstep": 2,
            "nrun" : 10000  # (max) number of  trajectories.
            }

In [ ]:
# Calculated variables
globalDict['m1'] = globalDict['rho']*(4/3)*np.pi*(globalDict['bhead']**2)*globalDict['ahead']
globalDict["m2"] = globalDict["rho"]*(4/3)*np.pi*(globalDict["bbutt"]**2)*globalDict["abutt"]
globalDict["echead"] = globalDict["ahead"]/globalDict["bhead"]
globalDict['ecbutt'] = globalDict['abutt']/globalDict['bbutt']
globalDict['I1'] = (1/5)*globalDict['m1']*(globalDict['bhead']**2)*(1 + globalDict['echead']**2)
globalDict['I2'] = (1/5)*globalDict['m2']*(globalDict['bbutt']**2)*(1 + globalDict['ecbutt']**2)
globalDict['S_head'] = np.pi*globalDict['bhead']**2
globalDict['S_butt'] = np.pi*globalDict['bbutt'] **2
globalDict["t"] = np.linspace(0, 0.02, num = globalDict["nstep"], endpoint = True)

In [ ]:
# ranges for variables
rangeDict = {"Fmin": 0,
             "Fmax": 44300,
            "alphaMin":  0,
             "alphaMax":2*np.pi, 
            "tau0Min": -100000, 
             "tau0Max": 100000}

In [ ]:
dataDict = {**globalDict, **rangeDict}

In [53]:
import importlib
_ = importlib.reload(simUtils)

In [10]:
# ranges for initial conditions
ranges = np.array([[0, 44300], [0, 2*np.pi], [-100000, 100000]])


In [11]:
# generate random initial conditions for state 0
dataDict["state0_ICs"] = [0, 0.0001, 0, 0.0001, 0, 0.0001, 0, 0.0001]

dataDict["randomizedFAlaphTau"] = np.random.uniform(ranges[:, 0], ranges[:, 1], size=(dataDict["nrun"], ranges.shape[0]))

In [12]:
dataDict

{'bhead': 0.507,
 'ahead': 0.908,
 'bbutt': 0.1295,
 'abutt': 1.7475,
 'rho': 1,
 'rhoA': 0.00118,
 'muA': 0.000186,
 'L1': 0.908,
 'L2': 1.7475,
 'L3': 0.75,
 'K': 29.3,
 'c': 14075.8,
 'g': 980.0,
 'betaR': 0.0,
 'nstep': 2,
 'nrun': 10000,
 'm1': 0.9776656946819244,
 'm2': 0.12275673548335365,
 'echead': 1.7909270216962525,
 'ecbutt': 13.494208494208495,
 'I1': 0.2114716316905065,
 'I2': 0.07538556187102556,
 'S_head': 0.8075432500126029,
 'S_butt': 0.05268529419886423,
 't': array([0.  , 0.02]),
 'Fmin': 0,
 'Fmax': 44300,
 'alphaMin': 0,
 'alphaMax': 6.283185307179586,
 'tau0Min': -100000,
 'tau0Max': 100000,
 'state0_ICs': [0, 0.0001, 0, 0.0001, 0, 0.0001, 0, 0.0001],
 'randomizedFAlaphTau': array([[ 3.08535849e+04,  1.79786647e+00, -5.46297093e+04],
        [ 2.44232443e+04,  4.52055686e+00, -1.53787080e+04],
        [ 4.34478540e+04,  4.30291215e+00, -3.81361970e+03],
        ...,
        [ 1.20961742e+04,  2.99879578e+00, -5.61372873e+04],
        [ 3.88207564e+04,  9.66505624

In [13]:
stt = time.time()
ss = [simUtils.flyBug_dictInput(ii, dataDict) for ii in range(100)]
print(time.time() - stt)

13.593993425369263


In [31]:
import importlib
_ = importlib.reload(simUtils)

In [35]:
d1 = {"foo": 5, "bar": 10}

In [36]:
d2 = {"bat": [10, 10, 50 ,10, 10 ,10]}
zip(d1, d2)

In [39]:
zip(dataDict, range(25))

In [43]:
# try to run simulations in paralell
from itertools import repeat
p = Pool(cpu_count() - 1)
stt = time.time()
ans = p.starmap(simUtils.flyBug_dictInput, zip(range(2500),repeat(dataDict)))
print(time.time() - stt)
p.close()
p.join()

13.386538028717041


In [32]:
#simUtils.flyBug_dictInput(0, dataDict)

# refref: numba failed to compile

In [44]:
# try to run simulations in paralell
p = Pool(cpu_count() - 1)
stt = time.time()
ans = p.map(functools.partial(simUtils.flyBug_dictInput, dataDict=dataDict), range(2500))
print(time.time() - stt)
p.close()
p.join()

13.306643724441528


In [54]:
simNum = 0
F = dataDict["randomizedFAlaphTau"][simNum][0]
alpha = dataDict["randomizedFAlaphTau"][simNum][1]
tau0 = dataDict["randomizedFAlaphTau"][simNum][2]
simUtils.FlyTheBug(dataDict["state0_ICs"], dataDict["t"], simNum, dataDict, F, alpha, tau0 )

array([ 1.00000000e-04, -6.31201016e+03,  1.00000000e-04,  1.42316015e+05,
        1.00000000e-04, -1.56637801e+05,  1.00000000e-04,  5.32637849e+04])

In [55]:
simUtils.FlyTheBug.inspect_types()

FlyTheBug (reflected list(int64), array(float64, 1d, C), int64, pyobject, float64, float64, float64)
--------------------------------------------------------------------------------
# File: C:\Users\calli\Documents\GitRepos\ODE_Python\PythonCode\simUtils.py
# --- LINE 9 --- 
# label 0

@jit

# --- LINE 10 --- 

def FlyTheBug(state, t, simNum, dataDict, F, alpha, tau0):

# --- LINE 11 --- 



    # --- LINE 12 --- 

    # unpack the state vector

    # --- LINE 13 --- 
    #   state = arg(0, name=state)  :: pyobject
    #   t = arg(1, name=t)  :: pyobject
    #   del t
    #   simNum = arg(2, name=simNum)  :: pyobject
    #   del simNum
    #   dataDict = arg(3, name=dataDict)  :: pyobject
    #   F = arg(4, name=F)  :: pyobject
    #   alpha = arg(5, name=alpha)  :: pyobject
    #   tau0 = arg(6, name=tau0)  :: pyobject
    #   $0.10 = exhaust_iter(value=state, count=8)  :: pyobject
    #   del state
    #   $0.2 = static_getitem(value=$0.10, index=0, index_var=None)  :: pyobject
    #

In [ ]:
len(bb)

In [ ]:
# save data to csv
newDF = pd.DataFrame(bb, columns= ["x0", "xf", "xd0", "xdf", "y0", "yf", "yd0", "ydf", "theta0", \
"thetaf", "thetad0", "thetadf", "phi0", "phif", "phid0", "phidf", "F", "alpha", "tau0"])


newDF.head()

In [ ]:
# save data to csv
newDF = pd.DataFrame(bb, columns= ["x0", "xf", "xd0", "xdf", "y0", "yf", "yd0", "ydf", "theta0", \
"thetaf", "thetad0", "thetadf", "phi0", "phif", "phid0", "phidf", "F", "alpha", "tau0"])


newDF.head()

In [ ]:
tstamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
print(tstamp)

In [ ]:
pythonMadeData

In [ ]:
newDF.to_csv(os.path.join(pythonMadeData, 
                          "RandomInitCond_" + tstamp + ".csv"),
             index = False)

In [ ]:
# for loop to run parallel processes a few times

for ii in range(8):
    # parallel run of simulations -- this generates approximately 1 million simulations per hour
    # this cell takes about an hour to run
    p = Pool(cpu_count()-2)
    stt = time.time()   
    bb = p.map(multiProcTraj.flyBug_firstLast, range(100000) ) 
    print(time.time() -stt)

    p.close()
    p.join()
    print(time.time() -stt)
    newDF = pd.DataFrame(bb, columns= ["x0", "xf", "xd0", "xdf", "y0", "yf", "yd0", "ydf", "theta0", \
            "thetaf", "thetad0", "thetadf", "phi0", "phif", "phid0", "phidf", "F", "alpha", "tau0"])
    
    tstamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
    print(tstamp)
    
    newDF.to_csv(os.path.join(pythonMadeData, 
                          "RandomInitCond_" + tstamp + ".csv"),
             index = False)

In [ ]:
# refref: one million simulations per 96 seconds?

In [ ]:
# for loop to run parallel processes a few times
for ii in range(8):
    # parallel run of simulations -- this generates approximately 1 million simulations per hour
    p = Pool(cpu_count()-4)
    stt = time.time()   
    bb = p.map(multiProcTraj.flyBug_firstLast, range(100000) ) 
    print(time.time() -stt)

    p.close()
    p.join()
    print(time.time() -stt)
    newDF = pd.DataFrame(bb, columns= ["x0", "xf", "xd0", "xdf", "y0", "yf", "yd0", "ydf", "theta0", \
            "thetaf", "thetad0", "thetadf", "phi0", "phif", "phid0", "phidf", "F", "alpha", "tau0"])
    
    tstamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
    print(tstamp)
    
    newDF.to_csv(os.path.join(pythonMadeData, 
                          "RandomInitCond_" + tstamp + ".csv"),
             index = False)

In [ ]:
# for loop to run parallel processes a few times

for ii in range(8):
    # parallel run of simulations -- this generates approximately 1 million simulations per hour
    # this cell takes about an hour to run
    p = Pool(cpu_count()-2)
    stt = time.time()   
    bb = p.map(multiProcTraj.flyBug_firstLast, range(100000) ) 
    print(time.time() -stt)

    p.close()
    p.join()
    print(time.time() -stt)
    newDF = pd.DataFrame(bb, columns= ["x0", "xf", "xd0", "xdf", "y0", "yf", "yd0", "ydf", "theta0", \
            "thetaf", "thetad0", "thetadf", "phi0", "phif", "phid0", "phidf", "F", "alpha", "tau0"])
    
    tstamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
    print(tstamp)
    
    newDF.to_csv(os.path.join(pythonMadeData, 
                          "RandomInitCond_" + tstamp + ".csv"),
             index = False)